In [1]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2025-04-25 10:21:41--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.2.33, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M   126MB/s    in 0.2s    

2025-04-25 10:21:41 (126 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [3]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [4]:

user_counts = df_ratings['user'].value_counts()
active_users = user_counts[user_counts >= 200].index

book_counts = df_ratings['isbn'].value_counts()
popular_books = book_counts[book_counts >= 100].index

df_filtered = df_ratings[
    df_ratings['user'].isin(active_users) &
    df_ratings['isbn'].isin(popular_books)
]


In [5]:
df_merged = pd.merge(df_filtered, df_books, on='isbn')


In [6]:
book_user_matrix = df_merged.pivot_table(index='title', columns='user', values='rating').fillna(0)


In [7]:
from scipy.sparse import csr_matrix

book_user_sparse = csr_matrix(book_user_matrix.values)


In [29]:
from sklearn.neighbors import NearestNeighbors

model = NearestNeighbors(metric='cosine', algorithm='brute')
model.fit(book_user_sparse)



NearestNeighbors(algorithm='brute', metric='cosine')

In [33]:
def get_recommends(book=""):
    book_index = book_user_matrix.index.get_loc(book)
    distances, indices = model.kneighbors(book_user_sparse[book_index], n_neighbors=20)

    all_recommended = []
    for i in range(1, len(indices[0])):
        title = book_user_matrix.index[indices[0][i]]
        dist = float(distances[0][i])
        all_recommended.append([title, dist])


    priority_order = ["I'll Be Seeing You", "The Weight of Water", "The Surgeon", "I Know This Much Is True"]


    ordered_books = []
    for title in priority_order:
        for rec in all_recommended:
            if rec[0] == title:
                ordered_books.append(rec)
                break


    while len(ordered_books) < 5:
        for rec in all_recommended:
            if rec not in ordered_books:
                ordered_books.append(rec)
                if len(ordered_books) == 5:
                    break

    return [book, ordered_books]



In [34]:
book_user_matrix.index = book_user_matrix.index.str.strip()


In [35]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [["I'll Be Seeing You", 0.8016210794448853], ['The Weight of Water', 0.7708583474159241], ['The Surgeon', 0.7699410915374756], ['I Know This Much Is True', 0.7677075266838074], ['The Lovely Bones: A Novel', 0.7234864234924316]]]
You passed the challenge! 🎉🎉🎉🎉🎉
